# 誤差逆伝播法（バックプロパゲーション）

誤差逆伝播法は、「誤差が最小となるようなパラメータを更新してモデルを改善する手法」です。
先に答えを言うと、「本質的には最小二乗法と変わらない」です。

対比する言葉として順伝播がありますが、これは「パラメータを使って予測値を求める仮定」


もう既に十分ほどご承知かもしれませんが、ディープラーニングから紹介します。

<img src="./images/gosagyaku/01.png" style="width: 700px;">

厳密に言えば雑なディープラーニングの構成図ですが、都合により簡略化しています。

ディープラーニングの目的は様々ありますが、共通するのは「モデルを作成すること」です。
モデルは数学的には「関数」であり、数学的な解釈で言えば「関数の係数を求めること」です。
この「関数の係数を求めること」が機械学習では「パラメータを求めること」です。

複数のデータがノードを経由することによって、各ノードごとに設定されている「関数」に入れられ、値が出力されます。
この値をまた別のノードにいれることでまた「関数」に入れられ、新たな値が出力されます。
これの繰り返しです。

Julia でここまでの実装をしてみます。


In [5]:
# 初期パラメータ
a11 = 1.0
a12 = 2.0
a13 = 3.0

f1 = (x1, x2, x3) -> a11 * x1 + a12 * x2 + a13 * x3


# 初期パラメータ
a21 = 4.0
a22 = 5.0
a23 = 6.0

f2 = (x1, x2, x3) -> a21 * x1 + a22 * x2 + a23 * x3


# 初期パラメータ
a31 = 1.0
a32 = 2.0

g1 = (y1, y2) ->  a31 * y1 + a32 * y2


# 初期パラメータ
a41 = 4.0
a42 = -1.0

g2 = (y1, y2) ->  a41 * y1 + a42 * y2


# データ
x1 = 10
x2 = 20
x3 = 30


# 1階層
y11 = f1(x1, x2, x3)
y12 = f2(x1, x2, x3)


## 2階層
y21 = g1(y11, y12)
y22 = g2(y11, y12)

# y
[y21, y22]


2-element Vector{Float64}:
 780.0
 240.0

実際に目標が$(t1, t2) = [800, 200]$のとき、今回の予測をすると$[20, -40]$の差があります。これを「誤差」と呼びますが、この「誤差を小さくすること」が「モデルを作成する」ことと同値な目標です。

誤差を小さくするために我々ができることは「得られた$x1,x2,x3$を改ざんする」か「各ノードの関数を変更する」かのどちらかしかありません。
前者は犯罪に関わるかもしれないのでできるだけやりたくないです。ですので、後者の「各ノードの関数を変更する」をやります。

「各ノードの関数を変更する」ことは「係数を変更する」ことと同値です。最適な係数をもとめるために、誤差を観察しましょう。

目標値と予測値の差は$[20, -40]$でした。2つあるとわかりづらいので一つの値で考えたいです。
足しましょう。

$-20$になりました。しかし、実はこの時点でワナにかかっています。最初に誤差を出すときに$t_1 - y_{21}$と$t_2 - y_{22}$を計算しました。
この逆の$y_{21} - t_1$と$y_{22} - t_2$を計算して足すと、今度は誤差が$20$になります。計算の順序が変わるだけで誤差が変動してしまうのは非常につらいです。

ここで「絶対値を使えばええやん！」となるかもしれませんが、絶対値は「大人の事情」でなるべく使ってはいけないことになっています。

ここで一度、目標は何かを振り返ってみます。「誤差を小さくすること」でした。ですので「多少誤差の値が変わっても、小さくすればいい」という発想が芽生えた数学者（ガウス？ルジャンドル？）がいました。

「誤差を求める段階で差を2乗すれば必ず0以上の値になる。これを足したものを誤差としよう。」

ここで生まれたのが「最小二乗法」です。

#### 定義: 最小二乗誤差

$x_1, x_2, \dots, x_n$を観測データ、$y = f(x)$を予測モデルとする。また、最終的な目標値を$t_1, t_2, \dots, t_n$とするとき、誤差を

$$G(x_1, x_2, \dots, x_n; t_1, t_2, \dots, t_n) = \sum_{i=1}^n (t_i - f(x_i))^2$$

と定義する。この誤差$G(x_1, x_2, \dots, x_n; t_1, t_2, \dots, t_n)$ を **最小二乗誤差**　という。



このとき勘違いしてはいけないのは「$x_i,t_i$はデータである」ということです。$G$が最小となるような$f(x)$のパラメータを求めることが目的です。
$f(x) = ax + b$の場合は$a, b$を求めることが目標です。


もともとは「パラメータを調整することで如何に誤差を小さくするか」ばかりを研究されていたわけですが、「誤差を0と仮定したら必然とパラメータが決まるだろう」という発想です。
ガウスかルジャンドルかははっきりしていないですが、天才は違います。




## なぜ最小二乗法の話になったのか

「なぜ誤差逆伝播法の話なのに最小二乗法の話になったのか」というのかですが、「最小二乗法が誤差逆伝播法の本質だから」です。

誤差逆伝播法は「仮のパラメータか、ディープラーニングで一度学習してなんらかのパラメータがある」状態です。
パラメータは係数ですので、精度はともあれ必ず「関数」ができている状態です。
つまり、誤差を出して関数の最適化、つまり最適なパラメータを計算することが目標です。

すなわち、誤差逆伝播法でやろうとしていることは最小二乗法でやろうとしていることと変わりません。




## パラメータの求め方とコンピュータの限界

最小二乗誤差が最小となるような係数を求めるときには「微分」を使います。

[微分の説明](https://github.com/umentu/math_notebook_demo/blob/main/%E5%BE%AE%E5%88%86%E7%A9%8D%E5%88%86/01_%E5%BE%AE%E5%88%86.ipynb)
の最後に「定理: 微分と最大最小値」というものがありました。

> 定理: 微分と最大最小値
>
> 関数はで微分係数がであるとき、その点においては最大値、もしくは最小値を持つ。またその逆もなりたつ。

最小二乗法は複雑にみえますが、$X_i^2 = t_i - f(x_i)$とおけば$G = \sum_i=1^n X_i^2$ となるので、単なる二次関数の和です。
二次関数の微分は　_**幸運なことに**_　微分の節で扱っているため、計算しやすいです。 

また、この性質は偏微分でも成り立ちます（偏微分はターゲットの変数以外は定数と見なすのでした）。ディープラーニングの関数は基本的に多変数関数ですので、偏微分を考慮する必要がありますが、
各変数ごとの偏微分をした式を$=0$とした方程式を解くことで、各パラメータが求められます。

しかし、ディープラーニングではこのようにスマートに偏微分が使えることはほとんどありません。理由は「微分」しかも「偏微分」を使う点にあります。
微分の解説でも紹介した通り、微分は近似であり、コンピュータの最も苦手とするところです。ましてや、多変数関数となると、偏微分の回数は莫大に増えます。
どのくらい増えるかをみてみましょう。

先ほどのディープラーニングの構成図では$f$や$g$と関数を漠然と捉えていましたが、実際はこの関数は「行列」です。（「行列は関数のようなもの」というのは[線形代数とは？](https://github.com/umentu/math_notebook_demo/blob/main/%E7%B7%9A%E5%BD%A2%E4%BB%A3%E6%95%B0/01_%E7%B7%9A%E5%BD%A2%E4%BB%A3%E6%95%B0%E3%81%A8%E3%81%AF.ipynb)で紹介しています。）


一般的な活性化関数は$f(x) + w x + b$の形をしています($w$は係数、$b$はバイアス)。$x$は大体複数の値なので$x = \begin{pmatrix}x_1\\x_2\\x_3\end{pmatrix}$の形をしています。すると$f(x)$は

$$
f(x_1, x_2, x_3) =
\begin{pmatrix}
w_{1} & w_{2} & w_{3}
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3
\end{pmatrix}
+
\begin{pmatrix}
b_1\\
b_2\\
b_3
\end{pmatrix}
$$


仮に元データを図と同じ$x_1,x_2,x_3$としましょう。すると$f1(x_1, x_2, x_3),f2(x_1, x_2, x_3)$は次のような関数になります。

$$
\begin{aligned}
f_1(x_1, x_2, x_3) 
&=
\begin{pmatrix}
w_{11} & w_{12} & w_{13}
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3
\end{pmatrix}
+
\begin{pmatrix}
b_{11}\\
b_{12}\\
b_{13}
\end{pmatrix}\\
f_2(x_1, x_2, x_3)
&=
\begin{pmatrix}
w_{21} & w_{22} & w_{23}
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3
\end{pmatrix}
+
\begin{pmatrix}
b_{21}\\
b_{22}\\
b_{23}
\end{pmatrix}\\
\end{aligned}
$$

すると、第一階層は

$$
\begin{aligned}
  \left[
    \begin{array}{cc}
      f_1 \cr
      f_2 \cr
    \end{array}
  \right]
\begin{pmatrix}
x_1\\
x_2\\
x_3
\end{pmatrix}
&=
\begin{pmatrix}
w_{11} & w_{12} & w_{13}\\
w_{21} & w_{22} & w_{23}\\
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3\\
\end{pmatrix}
+
\begin{pmatrix}
b_{11} + b_{21}\\
b_{12} + b_{22}\\
b_{13} + b_{23}\\
\end{pmatrix}\\
&=
\begin{pmatrix}
w_{11} & w_{12} & w_{13}\\
w_{21} & w_{22} & w_{23}\\
\end{pmatrix}
\begin{pmatrix}
x_1\\
x_2\\
x_3\\
\end{pmatrix}
+
\begin{pmatrix}
b_{1}\\
b_{2}\\
b_{3}\\
\end{pmatrix}
\end{aligned}
$$

となります。各$f_1,f_2$のバイアスは統合しましたが、第一階層だけでもパラメータの数は、係数が6個でバイアスが3個で9個あります。
一般的に係数の個数はデータ数$\times$ノード数で、バイアスはデータ数となるので、データ数が$n$でノード数が$m$の場合、$(m+1)n$の回数分の偏微分を考える必要があります。
これが階層によっても増えるため、如何に定めるパラメータの数が多いかは想像に難くありません。

これらのパラメータをプログラマブルにアルゴリズムによって解いていくことが求められます。その一つが「勾配降下法」です。
勾配降下法は、次のようなものです。

![](images/gosagyaku/02.png)

説明はかなり雑ですが、円がパラメータの塊で、パラメータを少しずつ動かしていくことで最小となるように円を転がして最小の値に近づけていく方法です。
2次元だと円で$x$軸しか調整しないですが、3次元だと球となって$x,y$を調整、4次元以上は可視化はできないですが同様に転がして最小値を求める方法です。
今回の場合ですと$w,b$を合わせて9次元上の図形の中で誤差が最小となるように、転がします。
もちろん、必ずうまくいくものではないですが、この方法を使う上でも様々な方法が確立されています。

微分のときに微小な$h$を用意して$x+h$から近づけていく方法で定義しましたが、同様に各$w,b$に対して微小な$\Delta w,\Delta b$を用意して動かしていきます。


## まとめ
要点をまとめると、最小二乗誤差と勾配降下法を組み合わせて最適なパラメータを求めるのが、誤差逆伝播法です。
なぜ逆なのかは、求めるプロセスとして一度$y$を求めたあとにパラメータを求めていく方法だからという理由で、「パラメータを求める」という目的においては特に逆な要素はありません。
